In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import re
import spacy
df = df.fillna('')
nlp = spacy.load('en_core_web_md')
comma_pattern = re.compile(r"(.+?)\,\s*(.+)$")
df["keyword"] = df["keyword"].apply(lambda x: x.replace('%20', ' '))
df["keyword_lemma"] = df["keyword"].apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
df["location_prefix"] = df["location"].apply(lambda x:  comma_pattern.search(x).group(1).strip() if comma_pattern.search(x) else x.strip())
df["location_lower_without_space"] = df["location_prefix"].apply(lambda x: x.replace(' ', '').lower())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import SVC
import numpy as np
def shuffled_data(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

svc_clf = SVC()

In [ ]:
prefix_vec = DictVectorizer()
lprefix_dict = df[['keyword', 'location_prefix']].to_dict(orient = 'record')
lprefix_vector = prefix_vec.fit_transform(lprefix_dict).toarray()
lprefix_x, lprefix_y = shuffled_data(lprefix_vector, df.target.to_numpy())

In [ ]:
svc_clf.fit(lprefix_x, lprefix_y)

In [ ]:
import pandas as pd 
df_test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
df_test

In [ ]:
df_test = df_test.fillna('')
#nlp = spacy.load('en_core_web_md')
comma_pattern = re.compile(r"(.+?)\,\s*(.+)$")
df_test["keyword"] = df_test["keyword"].apply(lambda x: x.replace('%20', ' '))
df_test["location_prefix"] = df_test["location"].apply(lambda x:  comma_pattern.search(x).group(1).strip() if comma_pattern.search(x) else x.strip())

In [ ]:
lprefix_dict_test = df_test[['keyword', 'location_prefix']].to_dict(orient = 'record')
test_x = prefix_vec.transform(lprefix_dict_test).toarray()

In [ ]:
test_x.shape

In [ ]:
results = svc_clf.predict(test_x)

In [ ]:
df_test['predict'] = results

In [ ]:
from collections import Counter
Counter(df_test['predict'])

In [ ]:
df_test

In [ ]:
import pandas as pd 
df_sub = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
df_sub['target'] = results

In [ ]:
df_sub.to_csv('SVM_submission.csv')

In [ ]:
ls

In [ ]:
from IPython.display import FileLink
FileLink(r'SVM_submission.csv')